# Medical Image - TFM
<h4>subtitle: Generación de una tubería distribuida para la extracción de características en imágenes médicas patológicas</h4>
license: European Union Public Licence (EUPL) v1.2

<table>
  <tr> <td> author name: </td> <td> Israel Llorens </td> </tr>
  <tr> <td> email: </td> <td> sanchezis@hotmail.com </td> </tr>
</table>

<h7>date: 2025/03/23</h7>

---

In [1]:
%load_ext autoreload
%autoreload

## Inicialización

In [2]:
import init
from digital_pathology.spark import spark

25/03/24 16:29:34 WARN Utils: Your hostname, Airon.local resolves to a loopback address: 127.0.0.1; using 192.168.100.210 instead (on interface en0)
25/03/24 16:29:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/24 16:29:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/24 16:29:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
import os
try:
    import pyspark
    from pyspark.sql.functions import col, isnan, when, count,to_date,year,month,expr,hour,dayofweek,lower,array_remove,collect_list,lit
    from pyspark.sql.functions import pandas_udf,split
    from pyspark.sql.types import StructField,StructType,StringType,DoubleType,FloatType,IntegerType, ArrayType, DoubleType, LongType
    import pyspark.sql.functions as F
except:
    pass
import pandas as pd

In [4]:
# Copyright (c) 2024 Israel Llorens
# Licensed under the EUPL-1.2  

__author__ = "Israel Llorens <sanchezis@hotmail.com>"
__copyright__ = "Copyright 2024, Israel Llorens"
__license__ = "EUPL-1.2"

import logging
import os

import pandas as pd
import glob
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

from tiatoolbox import logger
from tiatoolbox.models.architecture.unet import UNetModel
from tiatoolbox.models.engine.semantic_segmentor import (
    IOSegmentorConfig,
    SemanticSegmentor,
)
from tiatoolbox.utils.misc import download_data, imread
from tiatoolbox.utils.visualization import overlay_prediction_mask
from tiatoolbox.wsicore.wsireader import WSIReader

try:
    import pyspark
    from pyspark.sql.functions import col, isnan, when, count,to_date,year,month,expr,hour,dayofweek,lower,array_remove,collect_list,lit
    from pyspark.sql.functions import pandas_udf,split
    from pyspark.sql.types import ArrayType, DoubleType, StringType
    from pyspark.sql.types import StructField,StructType,StringType,DoubleType,FloatType,IntegerType, LongType
    import pyspark.sql.functions as F
except:
    pass


# Clear logger to use tiatoolbox.logger
import logging
import warnings

if logging.getLogger().hasHandlers():
    logging.getLogger().handlers.clear()

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch
from matplotlib import cm

from tiatoolbox import logger
from tiatoolbox.models.architecture.unet import UNetModel
from tiatoolbox.models.engine.semantic_segmentor import (
    IOSegmentorConfig,
    SemanticSegmentor,
)
from tiatoolbox.utils.misc import download_data, imread
from tiatoolbox.utils.visualization import overlay_prediction_mask
from tiatoolbox.wsicore.wsireader import WSIReader

from urllib import request
import certifi
import ssl
import os

import numpy as np
import histomicstk as htk
import skimage
import scipy as sp

import init
from digital_pathology.spark import spark
from digital_pathology.process.model_selection import ResNetModel

## Gold Data

In [5]:
glob.glob(os.path.join('..','data', '*.parquet') )

['../data/3-nucleotids.parquet',
 '../data/2-stardist-nucleotids.parquet',
 '../data/0-extract.parquet',
 '../data/1-download.parquet',
 '../data/4-nucleotids.parquet']

In [9]:
data_0 = spark.read.parquet(os.path.join('..','data', '0-extract.parquet'))
data_1 = spark.read.parquet('../data/1-download.parquet')
data_3 = spark.read.parquet('../data/3-nucleotids.parquet')
data_4 = spark.read.parquet('../data/4-nucleotids.parquet')\
            .withColumnRenamed('extract_tumor(filename, data/patient_extracts)', 'features')

In [8]:
data_3

modificationTime,length,filename,patient_key,patient_id,found_nucleotids
2025-03-23 03:47:...,1333915,patient_010_node_...,patient_010_node_...,patient_010,144
2025-03-23 03:46:...,716617,patient_010_node_...,patient_010_node_...,patient_010,224
2025-03-23 03:47:...,388017,patient_010_node_...,patient_010_node_...,patient_010,14
2025-03-23 02:26:...,17166,patient_004_node_...,patient_004_node_...,patient_004,7
2025-03-23 02:38:...,696266,patient_005_node_...,patient_005_node_...,patient_005,32
2025-03-23 01:59:...,59455,patient_000_node_...,patient_000_node_...,patient_000,3
2025-03-23 03:14:...,106998,patient_008_node_...,patient_008_node_...,patient_008,68
2025-03-23 01:47:...,655083,patient_000_node_...,patient_000_node_...,patient_000,394
2025-03-23 03:47:...,449542,patient_010_node_...,patient_010_node_...,patient_010,187
2025-03-23 03:47:...,642386,patient_010_node_...,patient_010_node_...,patient_010,168


In [10]:
data_4.show(truncate=False)

+---------------------------------------------------------------+----------------------+-----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|filename                                                       |patient_key           |patient_id |features                                                                                                                                              |
+---------------------------------------------------------------+----------------------+-----------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|patient_000_node_1.tif_tile_44_x0_y166912_score22950.0.png     |patient_000_node_1.tif|patient_000|{[ 28738  32662      0      0 987176], [('Tumour', 2.7407), ('Stroma', 3.1149), ('Inflamatory', 0.0), ('Necrosis', 0.0), ('Others', 94.1444)]}  

In [12]:
data_4.printSchema()

root
 |-- filename: string (nullable = true)
 |-- patient_key: string (nullable = true)
 |-- patient_id: string (nullable = true)
 |-- features: struct (nullable = true)
 |    |-- bins: string (nullable = true)
 |    |-- out: string (nullable = true)



In [19]:
percentages = data_4\
    .select('filename', 'patient_key', 'patient_id',  'features.bins', 'features.out')\
    .withColumnRenamed('bins', 'bins')\
    .withColumnRenamed('out', 'percentages')
percentages    .show(truncate=False, n=10)

+---------------------------------------------------------------+----------------------+-----------+-----------------------------------------+-------------------------------------------------------------------------------------------------------+
|filename                                                       |patient_key           |patient_id |bins                                     |percentages                                                                                            |
+---------------------------------------------------------------+----------------------+-----------+-----------------------------------------+-------------------------------------------------------------------------------------------------------+
|patient_000_node_1.tif_tile_44_x0_y166912_score22950.0.png     |patient_000_node_1.tif|patient_000|[ 28738  32662      0      0 987176]     |[('Tumour', 2.7407), ('Stroma', 3.1149), ('Inflamatory', 0.0), ('Necrosis', 0.0), ('Others', 94.1444)] |
|patient_004

In [20]:
# Create a function to calculate sum of fees and fine
# as well as difference of fine and discount
def extract(fees, place):
    return eval(fees)[place][1]

schema =  DoubleType()

schema_1 = StructType([
        StructField("id", StringType(), False),
        StructField("num_list", DoubleType(), True)
    ])

# Call the user defined function inside UDF
# function to create new columns 
data_frame_udf = F.udf(extract, schema)

# Call the variable created through UDF in 
# previous step to add new columns
updated_data_frame = percentages\
                            .withColumn("Tumor", data_frame_udf(percentages["percentages"], F.lit(0)) / 100 )\
                            .withColumn("Stroma", data_frame_udf(percentages["percentages"], F.lit(1)) / 100 )\
                            .withColumn("Inflamatory", data_frame_udf(percentages["percentages"], F.lit(2)) / 100 )\
                            .withColumn("Necrosis", data_frame_udf(percentages["percentages"], F.lit(3)) / 100 )\
                            .withColumn("Others", data_frame_udf(percentages["percentages"], F.lit(4)) / 100 )\
                            .drop('percentages')

updated_data_frame.printSchema()

# Display the updated data frame with 
# multiple columns added using UDF
updated_data_frame.show(truncate=False)

root
 |-- filename: string (nullable = true)
 |-- patient_key: string (nullable = true)
 |-- patient_id: string (nullable = true)
 |-- bins: string (nullable = true)
 |-- Tumor: double (nullable = true)
 |-- Stroma: double (nullable = true)
 |-- Inflamatory: double (nullable = true)
 |-- Necrosis: double (nullable = true)
 |-- Others: double (nullable = true)

+---------------------------------------------------------------+----------------------+-----------+-----------------------------------------+---------------------+--------------------+-----------+--------+------------------+
|filename                                                       |patient_key           |patient_id |bins                                     |Tumor                |Stroma              |Inflamatory|Necrosis|Others            |
+---------------------------------------------------------------+----------------------+-----------+-----------------------------------------+---------------------+--------------------+

## Analyze

In [21]:
updated_data_frame.filter(F.col('Tumor')>.02).count(), updated_data_frame.count(), 

(39, 200)

In [22]:
updated_data_frame.filter(F.col('Tumor')>.02)

filename,patient_key,patient_id,bins,Tumor,Stroma,Inflamatory,Necrosis,Others
patient_000_node_...,patient_000_node_...,patient_000,[ 28738 32662 ...,0.027407,0.031149,0.0,0.0,0.9414440000000001
patient_009_node_...,patient_009_node_...,patient_009,[ 8132 4631 ...,0.020104999999999998,0.011449,0.0,0.0,0.968446
patient_005_node_...,patient_005_node_...,patient_005,[ 56016 339454 ...,0.053421,0.323729,0.0,0.001117,0.621734
patient_000_node_...,patient_000_node_...,patient_000,[ 29657 191967 ...,0.028283,0.18307400000000001,0.0,0.0,0.788643
patient_004_node_...,patient_004_node_...,patient_004,[ 15929 150484 ...,0.039381,0.372043,0.0,0.0,0.5885750000000001
patient_004_node_...,patient_004_node_...,patient_004,[ 27472 77525 ...,0.06791900000000001,0.191666,0.0,0.0,0.740415
patient_000_node_...,patient_000_node_...,patient_000,[ 25337 4053 ...,0.024163,0.003865,0.0,6.29E-4,0.971342
patient_010_node_...,patient_010_node_...,patient_010,[ 30357 587290 ...,0.028950999999999998,0.560083,0.0,0.0,0.410966
patient_005_node_...,patient_005_node_...,patient_005,[ 60508 220960 ...,0.057705,0.210724,0.0,0.0,0.731571
patient_010_node_...,patient_010_node_...,patient_010,[ 22584 356687 ...,0.021537999999999998,0.340163,0.0,0.0,0.6382990000000001
